GGUF FILE LINK: https://huggingface.co/Tatvajsh/AHS_OPS_GGUF_V_1.0

In [ ]:
!pip install langchain ctransformers sentence-transformers faiss-gpu faiss-cpu

In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python --force-reinstall --upgrade --no-cache-dir


In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader,DirectoryLoader,TextLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.embeddings import LlamaCppEmbeddings

DATA_PATH="/content/data"
DB_FAISS_PATH="/content/vectorstores/db_faiss"

def create_vector_db():
    #Create a DirectoryLoader to load all PDFs from the DATA_PATH. Use the PyPDFLoader to load each PDF.
    loader=DirectoryLoader(DATA_PATH,glob="*.txt",loader_cls=TextLoader)
    documents=loader.load()
    #shared overlapping text gives some continuity between chunks and context.
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=80)
    # text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=150)

    texts=text_splitter.split_documents(documents)# all the splitted text is here,text chunks
    embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
                                     model_kwargs={'device':'cuda'})#creating the embeddings
    db=FAISS.from_documents(texts,embeddings)#using this embedding model,create all the embedding and store it
    db.save_local(DB_FAISS_PATH)

if __name__=="__main__":
        create_vector_db()

In [1]:
from llama_cpp import Llama
from langchain import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import CTransformers
from langchain.chains import RetrievalQA
llm = Llama(model_path="/content/drive/MyDrive/ggml-model-q8_0.gguf",n_gpu_layers=100)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
                                       model_kwargs={'device': 'cuda'})
DB_FAISS_PATH = '/content/vectorstores/db_faiss'
db = FAISS.load_local(DB_FAISS_PATH, embeddings)

# for out in output:
#     completionfragement=copy.deepcopy(out)
#     print(completionfragement["choices"][0]["text"])

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [24]:
question="What resources are available for the MOC Outpatient COVID Treatment Program?"
docs = db.similarity_search(question,k=1)
context_list=[]
for i in docs:
    context_list.append(i.page_content)
context="\n".join(context_list)
# print(context)
prompt=f"""Below is an instruction that describes a task. Write a response that appropriately completes the request from the given context
    ### Instruction:
    {question}
    ### Context:
    {context}
    ### Response:
    """
output = llm(prompt, max_tokens=1024,temperature=0.10, stop=["</s>"], echo=True)
# print(output)

Llama.generate: prefix-match hit


In [34]:
print(output["choices"][0]["text"].split("Response:")[1])


    The following resources are available for COVID treatment program at Alberta Health Services (AHS) MOC Outpatient Clinic in Edmonton, AB Canada:<br>
    <ul>
        <li>10 beds</li>
        <li>24/7 coverage by a physician and nurse practitioner</li>
        <li>Patients are seen on an appointment basis only.</li>
        <li>Patients must be referred to the clinic from their family physician or other health care provider. Patients can self-refer if they have been diagnosed with COVID-19, but will need a referral from their primary care provider before being seen at MOC Outpatient Clinic</li>
        <li>Patients are not able to be admitted directly into the clinic.</li>
        <li>Patients must be medically stable and able to travel to Edmonton for treatment. Patients who require admission to the hospital will be referred to the appropriate facility.</li>
        <li>Patients with a fever of 38C or higher, cough, shortness of breath, or other respiratory symptoms should not com